# Aula 10 - Recomendação baseada em sessão - Exercícios

In [1]:
import pandas as pd
import numpy as np

### Leitura do arquivo 2019-Oct-sample.csv (vide Aula 10 - Exemplos) caso não possua o arquivo

In [2]:
subset = pd.read_csv('2019-Oct-sample.csv')
subset.head()

,event_time,event_type,product_id,category_code,brand,user_session
0,2019-10-31 06:23:12 UTC,view,1005115,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c
1,2019-10-31 06:23:52 UTC,view,1005105,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c
2,2019-10-31 06:25:30 UTC,view,1005105,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c
3,2019-10-31 06:26:58 UTC,view,1004858,electronics.smartphone,samsung,00000056-a206-40dd-b174-a072550fa38c
4,2019-10-31 06:28:21 UTC,view,1005104,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c


In [3]:
map_items = {item: idx for idx, item in enumerate(subset.product_id.unique())}
map_sessions = {item: idx for idx, item in enumerate(subset.user_session.unique())}
subset['itemId'] = subset['product_id'].map(map_items)
subset['sessionId'] = subset['user_session'].map(map_sessions)
subset.head()

,event_time,event_type,product_id,category_code,brand,user_session,itemId,sessionId
0,2019-10-31 06:23:12 UTC,view,1005115,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c,0,0
1,2019-10-31 06:23:52 UTC,view,1005105,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c,1,0
2,2019-10-31 06:25:30 UTC,view,1005105,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c,1,0
3,2019-10-31 06:26:58 UTC,view,1004858,electronics.smartphone,samsung,00000056-a206-40dd-b174-a072550fa38c,2,0
4,2019-10-31 06:28:21 UTC,view,1005104,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c,3,0


In [4]:
n_items = subset['itemId'].max()+1
print('No. items: ', n_items)
n_sessions = subset['sessionId'].max()+1
print('No. sessions: ', n_sessions)

No. items:  42581
No. sessions:  483508


In [5]:
# create a dataset
# remove sessions with less than 2 items
def create_data(df):
    df.sort_values(by=['sessionId', 'event_time'], inplace=True, ignore_index=True)
    sessions, session = [], []
    for index, value in df.iterrows():
        if index != 0:
            if value["sessionId"] == df.at[index-1, "sessionId"]:
                if value["event_type"] == 'view':
                    session.append(value["itemId"])
            else:
                if len(session) > 1:
                    sessions.append((df.at[index-1, "sessionId"], session))
                session = [value["itemId"]]
        else:
            session.append(value["itemId"])
    return sessions

In [6]:
sessions = create_data(subset)

In [7]:
import random

random.shuffle(sessions)
split = len(sessions) * 0.8
train = sessions[:int(split)]
test = sessions[int(split):]
print('No. train sessions: ', len(train))
print('No. test sessions: ', len(test))

No. train sessions:  237531
No. test sessions:  59383


In [8]:
def jaccard(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [9]:
actual_session = test[37]
target = actual_session[1][0:-1]
print(actual_session)
print(target)
subset.loc[subset.sessionId==actual_session[0]]

(np.int64(107020), [1114, 71, 71])
[1114, 71]


,event_time,event_type,product_id,category_code,brand,user_session,itemId,sessionId
442858,2019-10-21 05:55:38 UTC,view,1005177,electronics.smartphone,samsung,04464488-826d-43e6-ad8d-4776452ad36f,1114,107020
442859,2019-10-21 06:01:33 UTC,view,1005217,electronics.smartphone,xiaomi,04464488-826d-43e6-ad8d-4776452ad36f,71,107020
442860,2019-10-21 06:03:48 UTC,view,1005217,electronics.smartphone,xiaomi,04464488-826d-43e6-ad8d-4776452ad36f,71,107020


In [10]:
def compute_score(train, target, itemId):
    candidate_sessions = []
    for s in range(len(train)):
        if itemId in train[s][1]:
            candidate_sessions.append(train[s][1])
    
    score = 0
    for n in range(len(candidate_sessions)):
        score += jaccard(candidate_sessions[n], target)
    
    return score


In [11]:
categories = subset.loc[subset.sessionId==actual_session[0]]['category_code'].unique().tolist()
candidate_items = subset.loc[subset.category_code.isin(categories)]['itemId'].unique().tolist()
len(candidate_items)

1190

In [12]:
ranking = []
for i in range(len(candidate_items)):
    ranking.append((compute_score(train, target, candidate_items[i]), candidate_items[i]))

ranking.sort()
ranking.reverse()
print(ranking[0:10])

[(134.60741237204738, 71), (35.74264886414714, 1114), (26.435962702008712, 297), (16.771315477668306, 29), (14.884769569350514, 31), (14.366276158167803, 985), (13.875696993449973, 30), (9.214863134349192, 90), (9.091980343687768, 2208), (9.066905370886047, 2368)]


In [13]:
subset.loc[subset.itemId==21083]

,event_time,event_type,product_id,category_code,brand,user_session,itemId,sessionId
262378,2019-10-28 11:56:34 UTC,view,21408343,electronics.clocks,romanson,02880533-d7ce-45bc-bc70-5d348203de11,21083,63162
329712,2019-10-11 15:52:48 UTC,view,21408343,electronics.clocks,romanson,032e7ca0-c78a-4167-b40b-e695e86a2f03,21083,79442
361345,2019-10-08 18:25:15 UTC,view,21408343,electronics.clocks,romanson,037aa64f-9798-4a10-b692-6d60d757ba6b,21083,87016
493880,2019-10-30 13:46:25 UTC,view,21408343,electronics.clocks,romanson,04c22360-b6c7-47bb-af50-501b85ce3399,21083,119192
657797,2019-10-08 17:07:25 UTC,view,21408343,electronics.clocks,romanson,0658adfb-8f43-4869-8f68-f11e16a40e45,21083,158819
667941,2019-10-09 07:40:36 UTC,view,21408343,electronics.clocks,romanson,06725817-89ff-4b8f-8c1a-adda02bd13a8,21083,161355
714952,2019-10-15 16:32:32 UTC,view,21408343,electronics.clocks,romanson,06e60861-aecc-4544-866b-a01499926aae,21083,172626
714953,2019-10-15 16:33:16 UTC,view,21408343,electronics.clocks,romanson,06e60861-aecc-4544-866b-a01499926aae,21083,172626
909150,2019-10-19 01:05:03 UTC,view,21408343,electronics.clocks,romanson,08c3c7ba-3874-45a7-a836-5952a02ed413,21083,219468
1055780,2019-10-07 10:35:35 UTC,view,21408343,electronics.clocks,romanson,0a3039ef-a7d3-4101-959b-30003cb161d4,21083,255199


## Para essa aula, você poderá escolher um dentre os três exercícios abaixo para resolver.

***Exercício 01:*** A função compute_score() definida acima e explicada na aula, é a implementação do algoritmo Session-based KNN (S-KNN). Implemente uma variação da função que represente o algoritmo Sequential Session-based KNN (S-SKNN). Compare o desempenho de ambas as funções na recomendação do último item de uma sessão qualquer do conjunto de teste. Para fazer essa comparação, utilize a métrica Average Precision. 

In [14]:
def is_subsequence(sub_session, main_session):
    """
    Verifica se sub_session é uma subseq de main_session.
    [1, 3] é uma subsequência de [1, 2, 3, 4]. [3, 1] não é.
    """
    
    it = iter(main_session)
    return all(item in it for item in sub_session)

In [15]:
def compute_score_ssknn(train, target, itemId):
    """
    Calcula o score para um itemId usando o algoritmo S-SKNN para apenas sessões vizinhas onde a target é uma subseq
    """
    
    candidate_sessions = []
    
    # Encontra as sessões de treino que têm o item candidato
    for s in range(len(train)):
        if itemId in train[s][1]:
            candidate_sessions.append(train[s][1])
    
    score = 0
    
    # Para cada candidato verifica a regra de sequência
    for n in range(len(candidate_sessions)):
        
        # Verifica se a target é uma subseq da sessão vizinha
        if is_subsequence(target, candidate_sessions[n]):
            score += jaccard(candidate_sessions[n], target)
    
    return score

In [16]:
def average_precision(ranking, actual_item):
    """
    Calcula a AP para um único item certo
    'ranking' é a lista de itemIds recomendado order por score.
    'actual_item' é o itemId que o usuário realmente viu/comprou.
    """
    # Inicializa a soma das precisões
    for i, recommended_item in enumerate(ranking):
        
        if recommended_item == actual_item:
            return 1.0 / (i + 1)
        
    # Se o item correto não estiver na lista, a AP é 0
    return 0.0

In [20]:
# Escolhe uma sessão do conjunto de teste
# Usar a mesma sessão do exemplo: test[37]
actual_session = test[37]
target = actual_session[1][:-1]  # Pega tudo menos o último item
actual_last_item = actual_session[1][-1] # Pega apenas o último item

print(f"Sessão de Teste (ID: {actual_session[0]}): {actual_session[1]}")
print(f"Sessão Alvo (Target): {target}")
print(f"Item Correto (Ground Truth): {actual_last_item}\n")

# Obter os itens candidatos (mesma estratégia da aula)
categories = subset.loc[subset.sessionId==actual_session[0]]['category_code'].unique().tolist()
candidate_items = subset.loc[subset.category_code.isin(categories)]['itemId'].unique().tolist()

# Ranking com S-KNN (original)
print("* Scores S-KNN original")

ranking_sknn = []

for item in candidate_items:
    ranking_sknn.append((compute_score(train, target, item), item))
    
ranking_sknn.sort(reverse=True) # Ordena do maior p/ menor score
ranked_items_sknn = [item for score, item in ranking_sknn]

# Ranking com S-SKNN seq
print("* Scores S-SKNN (sequencial)")

ranking_ssknn = []

for item in candidate_items:
    ranking_ssknn.append((compute_score_ssknn(train, target, item), item))
    
ranking_ssknn.sort(reverse=True) # Ordena do maior p/ menor score
ranked_items_ssknn = [item for score, item in ranking_ssknn]

# Average Precision dos algoritmos
ap_sknn = average_precision(ranked_items_sknn, actual_last_item)
ap_ssknn = average_precision(ranked_items_ssknn, actual_last_item)

Sessão de Teste (ID: 107020): [1114, 71, 71]
Sessão Alvo (Target): [1114, 71]
Item Correto (Ground Truth): 71

* Scores S-KNN original
* Scores S-SKNN (sequencial)


In [21]:
# Average Precision
print(f"AP com S-KNN: {ap_sknn:.4f}")
print(f"AP com S-SKNN: {ap_ssknn:.4f}\n")

# Top 5 de cada recomendação
print(f"Top 5 das recomendações")
print(f"S-KNN: {ranked_items_sknn[:5]}")
print(f"S-SKNN: {ranked_items_ssknn[:5]}")
print(f"O item correto era pra ser: {actual_last_item}")

AP com S-KNN: 1.0000
AP com S-SKNN: 0.3333

Top 5 das recomendações
S-KNN: [71, 1114, 297, 29, 31]
S-SKNN: [1114, 297, 71, 29, 6968]
O item correto era pra ser: 71


***Exercício 02:*** Implemente outra variação da função compute_score() que represente o algoritmo Sequential Filter Session-based KNN (SF-SKNN). Compare o desempenho desse algoritmo com as demais abordagens para uma sessão qualquer do conjunto de teste.

In [18]:
# TODO

***Exercício 03:*** Na aula utilizamos uma estratégia trivial para selecionar itens candidatos para poder calcular seu escore: selecionamos apenas itens da mesma categoria que os itens que estão na sessão atual. Isso pode ser um problema, pois numa sessão, um usuário pode estar visualizando um produto e o sistema poderia recomendar um produto de outra categoria (exemplo: usuário visualiza/compra um smartphone, e o sistema recomenda uma capa protetora). Pense e implemente uma estratégia para selecionar os itens candidatos para os quais será calculado o escore via função compute_score(). Lembre-se de que quanto menor a quantidade de itens candidatos, mais rápido o sistema irá gerar a recomendação top N. Explique sua estratégia.

In [19]:
# TODO